In [1]:
import numpy as np
from pyntcloud import PyntCloud
import os
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import tensorflow as tf
from keras import optimizers
from keras.layers import Input
from keras.models import Model
from keras.layers import Dense, Reshape
from keras.layers import Convolution1D, MaxPooling1D, BatchNormalization
from keras.layers import Lambda, concatenate
import h5py


k=17
adam = optimizers.Adam(lr=0.001, decay=0.7)

Using TensorFlow backend.


In [2]:
def dense_to_one_hot(labels_dense, num_classes):
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [3]:
def mat_mul(A,B):
    return tf.matmul(A,B)

In [4]:
def exp_dim(global_feature, num_points):
    return tf.tile(global_feature, [1, num_points, 1])

In [5]:
train=PyntCloud.from_file('/home/rahul/machine learning/Projects/rueMadame_database/GT_Madame1_3.ply')
num_points = train.points.shape[0]
print(train)



PyntCloud
10000000 points with 3 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -12.104286193847656, -197.56027221679688, 46.1270637512207
Other attributes:



In [6]:
test=PyntCloud.from_file('/home/rahul/machine learning/Projects/rueMadame_database/GT_Madame1_2.ply')
num_points = test.points.shape[0]
print(test)


PyntCloud
10000000 points with 3 scalar fields
0 faces in mesh
0 kdtrees
0 voxelgrids
Centroid: -7.090676784515381, -124.74476623535156, 45.12932205200195
Other attributes:



In [7]:
#input_transformation_net
input_points = Input(shape=(num_points, 3))
x = Convolution1D(64, 1, activation='relu',input_shape=(num_points, 3))(input_points)
x = BatchNormalization()(x)
x = Convolution1D(128, 1, activation='relu')(x)
x = BatchNormalization()(x)
x = Convolution1D(1024, 1, activation='relu')(x)
x = BatchNormalization()(x)
x = MaxPooling1D(pool_size=num_points)(x)
x = Dense(512, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(256, activation='relu')(x)
x = BatchNormalization()(x)
x = Dense(9, weights=[np.zeros([256, 9]), np.array([1, 0, 0, 0, 1, 0, 0, 0, 1]).astype(np.float32)])(x)
input_T = Reshape((3, 3))(x)
#forward_net
g = Lambda(mat_mul, arguments={'B': input_T})(input_points)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(64, 1, input_shape=(num_points, 3), activation='relu')(g)
g = BatchNormalization()(g)
#feature_transformation_net
f = Convolution1D(64, 1, activation='relu')(g)
f = BatchNormalization()(f)
f = Convolution1D(128, 1, activation='relu')(f)
f = BatchNormalization()(f)
f = Convolution1D(1024, 1, activation='relu')(f)
f = BatchNormalization()(f)
f = MaxPooling1D(pool_size=num_points)(f)
f = Dense(512, activation='relu')(f)
f = BatchNormalization()(f)
f = Dense(256, activation='relu')(f)
f = BatchNormalization()(f)
f = Dense(64 * 64, weights=[np.zeros([256, 64 * 64]), np.eye(64).flatten().astype(np.float32)])(f)
feature_T = Reshape((64, 64))(f)

# forward net
g = Lambda(mat_mul, arguments={'B': feature_T})(g)
seg_part1 = g
g = Convolution1D(64, 1, activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(128, 1, activation='relu')(g)
g = BatchNormalization()(g)
g = Convolution1D(1024, 1, activation='relu')(g)
g = BatchNormalization()(g)

# global_feature
global_feature = MaxPooling1D(pool_size=num_points)(g)
global_feature = Lambda(exp_dim, arguments={'num_points': num_points})(global_feature)

# point_net_seg
c = concatenate([seg_part1, global_feature])
c = Convolution1D(512, 1, activation='relu')(c)
c = BatchNormalization()(c)
c = Convolution1D(256, 1, activation='relu')(c)
c = BatchNormalization()(c)
c = Convolution1D(128, 1, activation='relu')(c)
c = BatchNormalization()(c)
c = Convolution1D(128, 1, activation='relu')(c)
c = BatchNormalization()(c)
prediction = Convolution1D(k, 1, activation='softmax')(c)
'''
end of pointnet
'''

# define model
model = Model(inputs=input_points, outputs=prediction)
print(model.summary())


Instructions for updating:
`NHWC` for data_format is deprecated, use `NWC` instead
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 10000000, 3)  0                                            
__________________________________________________________________________________________________
lambda_1 (Lambda)               (None, 10000000, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_4 (Conv1D)               (None, 10000000, 64) 256         lambda_1[0][0]                   
__________________________________________________________________________________________________
batch_normalization_6 (BatchNor (None, 10000000, 64) 256         conv1d_4[0][0]                   
__________________________

In [8]:
train.points.drop(labels=["label","reflectance"],inplace=True,axis=1)
train_labels=train.points["class"]
train.points.drop(labels=["class"],inplace=True,axis=1)
train_points = train.points.values.reshape(-1, num_points, 3)
train_labels=dense_to_one_hot(train_labels,17)

In [9]:
test.points.drop(labels=["label","reflectance"],inplace=True,axis=1)
test_labels=test.points["class"]
test.points.drop(labels=["class"],inplace=True,axis=1)
test_points = test.points.values.reshape(-1, num_points, 3)
test_labels=dense_to_one_hot(test_labels,17)

In [10]:
train_points.shape

(1, 10000000, 3)

In [11]:
train_labels.shape

(10000000, 17)

In [12]:
test_points.shape

(1, 10000000, 3)

In [13]:
test_labels.shape

(10000000, 17)

In [14]:
train_labels = np.array([train_labels])

In [15]:
train_labels.shape

(1, 10000000, 17)

In [16]:
test_labels=np.array([test_labels])

In [17]:
test_labels.shape

(1, 10000000, 17)

In [18]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [18]:
# rotate and jitter point cloud every epoch
model.fit(train_points, train_labels, batch_size=1, epochs=50, shuffle=True, verbose=1)    

score = model.evaluate(test_points, test_label50, verbose=1)
print('Test loss: ', score[0])
print('Test accuracy: ', score[1])

ValueError: Error when checking target: expected conv1d_16 to have 3 dimensions, but got array with shape (10000000, 17)